In [ ]:
import time

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
import numpy as np
import torch

from dimp.utils import init_matplotlib


init_matplotlib()

%matplotlib widget

## Example Pannocchia 🌽

### Define the problem matrices, the initial state, the time horizon, and the LQR matrices.

In [ ]:
A = np.array([
    [-0.1, 0, 0],
    [0, -2, -6.25],
    [0, 4, 0]
])

B = np.array([[0.25], [2.0], [0.0]])

s0 = np.array([1.344, -4.585, 5.674])

T = 10.0

N = 1000

dt = T / N

Q = 1.0 * np.eye(3)
R = 0.1 * np.eye(1)

u_max = 1.0

### Create the cxvpy variables and the optimization problem

In [ ]:
s = [s0] + [cp.Variable(3, name=f"s_{i}") for i in range(N)]
u = [cp.Variable(1, name=f"u_{i}") for i in range(N)]

def create_pann_clqr(n: int = N, dt: float = dt):
    objective = cp.Minimize(
        cp.sum([cp.quad_form(s[i+1], Q) for i in range(n)]) * dt +
        cp.sum([cp.quad_form(u[i], R) for i in range(n)]) * dt
    )
    
    dynamics_constraints = [
        s[i+1] == s[i] + (A @ s[i] + B @ u[i]) * dt for i in range(n)
    ]
    
    input_limits = [
        cp.abs(u[i]) <= u_max for i in range(n)
    ]

    constraints = dynamics_constraints + input_limits

    problem = cp.Problem(objective, constraints)
    
    return problem

pann_clqr = create_pann_clqr()
assert pann_clqr.is_dpp()

### Study how the solution changes with different number of samples.

With $n=80$ samples the system is unstable.
From $n=160$ timesteps the OCP stabilizes the system.
However, increasing the number of timesteps further improves the solution.

In [ ]:
# Number of timesteps of the trajectory optimization problem.
interval = 80

fig, axs = plt.subplots(6, 2, figsize=(6.4, 9.6))

i = 0
for n in range(interval, N + 1, interval):
    dt = T / n
    pann_clqr = create_pann_clqr(n=n, dt=dt)

    start_time = time.time()
    pann_clqr.solve()
    solve_time = time.time() - start_time
    
    print(
        f"n = {n}\n"
        f"Optimal cost: {pann_clqr.objective.value:.4f}, "
        f"solve time meas: {solve_time:.4f} s, "
        f"solve time: {pann_clqr.solver_stats.solve_time:.4f} s\n"
    )
    
    if n > 6*interval:
        continue

    times = np.arange(n) * dt
    s_vec = np.array([s.value for s in s[1:n+1]])
    u_vec = np.array([u.value for u in u[:n]])

    axs[2*(i//2), i%2].plot(times, s_vec, label=['x', 'y', 'z'])
    axs[2*(i//2), i%2].set(
        xlabel='Time',
        ylabel='State',
        title=fr"$n={n}$",
    )

    axs[2*(i//2)+1, i%2].plot(times, u_vec)
    axs[2*(i//2)+1, i%2].set(
        xlabel='Time',
        ylabel='Input',
    )

    # axs[i//2, (i+1)%2].legend()

    i = i + 1

### Create the Parametrized CLQR Problem

In [ ]:
n = 200

deltas = [cp.Variable(1, name=f'deltas_{i}') for i in range(n)]
dts = [cp.Parameter(1, nonneg=True, name=f'dts_{i}') for i in range(n)]

def create_pann_param_clqr(n: int = N):
    objective = cp.Minimize(
        cp.sum([cp.quad_form(s[i+1], Q) * dts[i] for i in range(n)]) +
        cp.sum([cp.quad_form(u[i], R) * dts[i] for i in range(n)]) + 
        10**3 * cp.sum([cp.square(deltas[i] - dts[i]) for i in range(n)])
    )
    
    dynamics_constraints = [
        s[i+1] == s[i] + (A @ s[i] + B @ u[i]) * dts[i] for i in range(n)
    ]
    
    input_limits = [
        cp.abs(u[i]) <= u_max for i in range(n)
    ]
    
    timestep_constraints = [
        deltas[i] >= 0 for i in range(n)
    ] + [
        cp.sum(deltas[0:n]) == T
    ]

    constraints = dynamics_constraints + input_limits + timestep_constraints

    problem = cp.Problem(objective, constraints)
    
    return problem

pann_param_clqr = create_pann_param_clqr(n)
assert pann_param_clqr.is_dpp()

In [ ]:
def task_loss(sol, method="unscaled"):
    states = [sol[i] for i in range(n)]
    inputs = [sol[n+i] for i in range(n)]
    deltas = [sol[2*n+i] for i in range(n)]
    
    Q_th = torch.tensor(Q, dtype=torch.float32, device=states[0].device)
    R_th = torch.tensor(R, dtype=torch.float32, device=states[0].device)
    
    if method == 'unscaled':
        return sum([
            si.t() @ Q_th @ si for si in states
        ]) + sum([
            ui.t() @ R_th @ ui for ui in inputs
        ])
    if method == 'time scaled':
        return sum([
            deltas[i] * states[i].t() @ Q_th @ states[i] for i in range(n)
        ]) + sum([
            deltas[i] * inputs[i].t() @ R_th @ inputs[i] for i in range(n)
        ])
    if method == 'time scaled fixed length':
        return sum([
            deltas[i] * states[i].t() @ Q_th @ states[i] for i in range(n)
        ]) + sum([
            deltas[i] * inputs[i].t() @ R_th @ inputs[i] for i in range(n)
        ]) + 1e3 * (sum(deltas) - T)**2
    
    raise ValueError(f"Unknown method {method}")

In [ ]:
def plot_timegrid(deltas, x=None, ax=None):
    times = np.cumsum(deltas.tolist())

    if ax is None:
        fig, ax = plt.subplots()
    for t in times:
        ax.axvline(t, color='gray', linestyle='--', alpha=0.5)
    
    if x is not None:
        ax.plot(times, x)
        
    ax.set_xlabel("Time")

In [ ]:
def plot_colored(deltas, x, ax=None):
    times = np.cumsum(deltas)
    
    cmap = plt.get_cmap("viridis")
    norm = Normalize(vmin=np.min(deltas), vmax=np.max(deltas))

    if ax is None:
        fig = plt.figure()
        ax = plt.gca()
    
    for i in range(len(x)-1):
        # horizontal hold
        ax.hlines(x[i], times[i], times[i+1], 
                  colors=cmap(norm(deltas[i])), linewidth=2)
        # vertical jump
        ax.vlines(times[i+1], x[i], x[i+1], 
                  colors=cmap(norm(deltas[i])), linewidth=1)
        
    ax.set_xlabel("Time")

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label("deltas")
    
    return ax

### Training Loop

In [ ]:
def plot_training_res(sol, history):
    s_arr = np.array([s.detach().numpy().tolist() for s in sol[0:n]])
    u_arr = np.array([u.detach().numpy().tolist() for u in sol[n:2*n]])
    d_arr = np.concatenate(np.array([d.detach().numpy().tolist() for d in sol[2*n:3*n]]))

    fig, ax = plt.subplots(2, 2, figsize=(9.6, 7.2))
    ax[0, 0].plot([h['loss'] for h in history])

    ax[0, 0].set_xlabel("Epoch")
    ax[0, 0].set_ylabel("Loss")
    ax[0, 0].set_title("Loss Evolution")
    
    plot_colored(d_arr, u_arr, ax[1, 0])
    
    plot_timegrid(d_arr, s_arr, ax[0, 1])

    plot_timegrid(d_arr, u_arr, ax[1, 1])

    fig.set_constrained_layout(True)

In [ ]:
history  = []

for method in ["unscaled", "time scaled", "time scaled fixed length"]:
    n = 160

    pann_param_clqr = create_pann_param_clqr(n)
    cvxpylayer = CvxpyLayer(
        pann_param_clqr,
        parameters=dts[:n],
        variables=s[1:n+1] + u[:n] + deltas[:n],
    )

    dts_torch = [torch.nn.Parameter(torch.ones(1) * dt) for _ in range(n)]

    optim = torch.optim.Adam(dts_torch, lr=5e-4)

    # =========================================================================== #

    n_epochs = 100

    with torch.no_grad():
        dt = T / n
        for d in dts_torch:
            d.copy_(torch.ones(1) * dt)

    for epoch in range(n_epochs):
        print(f"Epoch {epoch+1}/{n_epochs}")
        
        optim.zero_grad()

        sol = cvxpylayer(*dts_torch)
        
        loss = task_loss(sol, method=method)
        loss.backward()

        optim.step()
        with torch.no_grad():
            for d in dts_torch:
                d.clamp_(min=1e-6)
                d *= T / sum(dts_torch)

        history.append({
            'method': method,
            'loss': loss.item(),
            'dts': [d.detach().numpy() for d in dts_torch]
        })

    plot_training_res(sol, [h for h in history if h['method'] == method])

In [ ]:
print(history)